In [5]:
import pandas as pd
import os
import re
from dataclasses import dataclass
import json
from pymongo import MongoClient
import sqlite3
from functions import get_current_project
from vars import CURRENT_PROJECT_FILE, DATA_DIR, JSON_READINGS_FILE, PROJECT_DIR, PROJECT_DIR_AUDIO, PROJECT_DIR_EXPORT, PROJECT_DIR_EXPORT_VERSES, PROJECT_DIR_EXPORT_CHAPTERS, PROJECT_CONFIG_FILE_NAME, PROJECT_DOWNLOADS_DIR, PROJECT_JSON_DIR, PROJECT_TEMP_DOWNLOADS_DIR, PROJECT_TRANSCRIPTS_DIR, PROJECT_TRANSCRIPTS_DIR, PROJECT_DOWNLOADS_DIR, PROJECT_TEMP_DOWNLOADS_DIR, PROJECT_CSV_DIR, CSV_SEGMENTS_FILE, CSV_SOURCES_FILE, CSV_SEARCHES_FILE

client = MongoClient('mongodb://localhost:27017/')
db = client['ephtc']

searches = {s["word"]: s["segments"] for s in db["searches"].find({})}
segments = {s["id"]: s for s in db["segments"].find({})}

In [63]:
@dataclass
class Reference:
    book: str
    chapter: int
    verse: int
    content: str

@dataclass
class Reading:
    id: str
    start_time: float
    end_time: float
    start_seg: int
    end_seg: int

book = "Ephesians"

# get Bible verses
bible_sqlite = "ESV.sqlite"
with open("./references.json", "r") as f:
    references = json.load(f)

def get_esv_content(book: str, chapter: int, verse: int) -> str:
    con = sqlite3.connect(bible_sqlite)
    cur = con.cursor()
    res = cur.execute(
        f"SELECT content FROM '{book}' WHERE chapter={chapter} AND verse={verse} ORDER BY verse ASC;")
    element = res.fetchone()
    # print(verse)
    return element[0]

def get_all_verses_and_content() -> list[Reference]:
    refs: list[Reference] = []
    for chapter, verses_in_chapter in enumerate(references[book]):
        if chapter == 0:
            continue
        for verse in range(1, verses_in_chapter + 1):
            content = get_esv_content(book, chapter, verse)
            ref = Reference(book, chapter, verse, content)
            refs.append(ref)
    return refs


In [14]:
def print_segment_range(s, e):
    print(" ".join([segments[i]["content"] for i in range(s, e + 1)]))


In [77]:
ch = 4
v = 8
ref = Reference(book, ch, v, get_esv_content(book, ch, v))
valid_offset = 3
accuracy = 0.7
words = [word for word in re.split(r"\s", re.sub(r"[^A-z0-9\s]", "", ref.content.lower())) if len(word) > 0]
word_count = len(words)
print(words)

for i, word in enumerate(words[:2]):
    start_word = word
    end_word = words[-1 - i]
    start_segs = searches[start_word]
    end_segs = searches[end_word]
    # print(start_segs)
    # print(end_segs)
    # reading_segment_ranges = [(s, s + size) for s in start_segs if s + size in end_segs]
    # segments_index = [(s, e) for s, e in zip(start_segs, end_segs) if (e - s < valid_offset) and (s > e)]
    segments_index = [(s, e) for s in start_segs for e in end_segs if (s < e) and ((e - s - word_count) < valid_offset)]
    # print(segments_index)
    readings: list[Reading] = []
    for s, e in segments_index:
        valid_count = 0
        # print(s, e + 1)
        for j in range(s, e + 1):
            # print(j - s, words[j - s])
            if j - s >= word_count:
                continue
            valid_count += 1 if segments[j]["content"] == words[j - s] else 0
        rating = (valid_count) / (e - s)
        # valid segment -> give reading
        # print(rating)
        if rating > accuracy:
            readings.append(
                Reading(
                    segments[s]["source"],
                    segments[s]["start"],
                    segments[e]["end"],
                    s,
                    e
                )
            )
    if len(readings) > 0:
        break
print(readings)
for reading in readings:
    # print(s, e)
    print_segment_range(reading.start_seg, reading.end_seg)


['therefore', 'it', 'says', 'when', 'he', 'ascended', 'on', 'high', 'he', 'led', 'a', 'host', 'of', 'captives', 'and', 'he', 'gave', 'gifts', 'to', 'men']
[Reading(id='917181255551', start_time=69.777, end_time=75.761, start_seg=63841, end_seg=63860), Reading(id='12519162759462', start_time=211.065, end_time=218.332, start_seg=559464, end_seg=559483), Reading(id='121118172377422', start_time=320.128, end_time=329.612, start_seg=614086, end_seg=614105)]
therefore it says, when He ascended on high, He led a host of captives and He gave gifts to men.
therefore it says, when He ascended on high, He led a host of captives and He gave gifts to men.
Therefore it says when He ascended on high, He led a host of captives and He gave gifts to men.


In [43]:
print_segment_range(14,19)

Paul, an apostle of Christ Jesus


In [ ]:

        filters = []
        for s, e in segments_index:
            filters.append({"id": {"$gte": s, "$lte": e}})
        pipeline = [
            {
                "$match": {"$or": filters}
            }
        ]

        # Find documents matching the criteria
        results = collection.aggregate(pipeline)
